## 第二次课
1.前30分钟，检测大家的python水平（最好控制在30分钟左右）。

2.后30分钟，讲jieba/HanLP/nltk三个分词工具。

### 测验&作业

In [ ]:
# 测验：https://www.runoob.com/quiz/python-quiz.html

In [1]:
# 数据类型
# 题目：生成一个5位数，要求：一、求它是几位数，二、逆序打印出各位数字。
import random

a = random.randrange(10000, 99999) #生成随机数

length = len(str(a)) #获得位数

reverse = str(a)[::1] #作为字符串倒置五位数

li =[length, int(reverse)]
print(li)

# 最后输出要求：[5,25341]

[5, 12186]


In [2]:
# 字符串
# 题目：反转字符串，输入"hello"，输出"olleh"
string = "hello"

print(string[::-1]) #第一种

l = len(string)  #第二种
reverse = ''
for i in range(l-1, -1, -1):
    reverse = reverse+string[i]
print(reverse)

l = len(string)  #第三种
r = ''
i = 0
while i < l:
    r = r + string[l-1-i]
    i += 1
print(r)

# 最后要求输出："olleh"，使用三种方法。

olleh
olleh
olleh


In [4]:
# 循环
# 题目：有四个数字：1、2、3、4，能组成多少个互不相同且无重复数字的三位数？各是多少？
i,j,k,l = 1,2,3,4
li = [i, j, k, l]
result = []
for a in li:
    for b in li:
        for c in li:
            if a != b:
                if b != c:
                    if a != c:
                        num = 100*a+10*b+c
                        result.append(num)
print(result)
# 最后要求输出：[123,124,234,134,......]

[123, 124, 132, 134, 142, 143, 213, 214, 231, 234, 241, 243, 312, 314, 321, 324, 341, 342, 412, 413, 421, 423, 431, 432]


In [8]:
# 列表
# 题目：给定按相反的顺序输出列表的值，并且去重。
li = [4, 3, 2, 1, 1, 2, 3, 6, 2, 2, 3, 6, 2, 3, 7, 3, 3]
l = len(li)
li = li[::-1]
print(sorted(set(li), key = li.index))


# 最后要求输出：相反的list,且不能有重复。

[3, 7, 2, 6, 1, 4]


In [13]:
# 函数
# 给定一个name_list = ['adam', 'LISA', 'barT'],输出首字母大写，后续字母小写的list
def normalize(name):
    for i in range(len(name)):
        name[i] = name[i].capitalize()
    return name
name_list = ['adam', 'LISA', 'barT']
normalize(name_list)

# 最后要求输出name_list = ['Adam', 'List', 'Bart']

['Adam', 'Lisa', 'Bart']

In [32]:
# lambda函数
# 题目：请将列表[1,2,3,4,5]使用python方法转变成[1,4,9,16,25]。然后提取大于10的数，最终输出[16,25]
l = [1,2,3,4,5]

p = lambda a: a*a

def power(li):
    for i in range(len(li)):
        li[i] = p(li[i])
    return li

l = power(l)

new = []

for i in range(len(l)):
    if l[i] > 10:
        new.append(l[i])
        
print(new)

# 最后要求输出:[16,25]，必须使用lambda方法。

[16, 25]


### 算法使用部分

In [ ]:
import os
import jieba
from pyhanlp import *
import pandas as pd
from collections import Counter

In [ ]:
!pip install jieba

In [ ]:
jieba.add_word('贾宝玉')

In [ ]:
# 打开停用词词表
stopwords = []
with open("stopwords.txt","r",encoding = "utf-8") as f:
    for line in f.readlines():
        stopwords.append(line.strip("\n"))
        
# 载入红楼梦全本，或上面方法打开
red_chamber_table = pd.read_table("红楼梦.txt",sep='\t',header=None)
corpus = "".join(red_chamber_table[0])

# 精准模式切分
word_list = jieba.lcut(corpus)

# 清洗
word_list_clean = []
for word in word_list:
    if word not in stopwords:
        word_list_clean.append(word)
    
# 统计词频
freq_table = pd.DataFrame([dict(Counter(word_list_clean))])

In [ ]:
freq_table = freq_table.T
freq_table.columns = ["freq"]
freq_table.sort_values("freq",ascending = False).head(10)

In [62]:
# jieba除了精准模式（即默认），还有全切分模式和搜索切分模式
# 全切分：速度非常快，但不能解决歧义。
segs_all = jieba.cut(corpus, cut_all=True)
result = " ".join(segs_all)

# 可以写成lcut
# 搜索引擎模式：在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词
segs_4 = jieba.cut_for_search(corpus)
result = " ".join(segs_all)

In [63]:
# 获取词性
import jieba.posseg as psg
segs_5 = [(x.word,x.flag) for x in psg.lcut(corpus)]

In [28]:
# 思考问题：
"""
# 我们能不能按照章回，将所有文字切分成一个表格？
# 然后来分析每一回的用词，词频？
# 使用的最多的动词？使用的最多的名词？使用的最多的形容词？
"""

In [ ]:
## 下节课：正则表达式讲解

### 算法原理部分

In [9]:
import logging
import pandas as pd
import numpy as np
import jieba
import os

In [ ]:
####
# 以词典的为基础的算法，脱不开全切分、正向匹配/反向匹配/双向匹配四种。

In [66]:
# 全切分算法
from pyhanlp import *

def load_dictionary():
    IOUtil = JClass("com.hankcs.hanlp.corpus.io.IOUtil")
    path = HanLP.Config.CoreDictionaryPath.replace(".txt",".mini.txt")
    dic = IOUtil.loadDictionary([path])
    return set(dic.keySet())

def fully_segment(text,dic):
    word_list = []
    for i in range(len(text)):
        for j in range(i + 1,len(text) + 1):
            word = text[i:j]
            if word in dic:
                word_list.append(word)
    return word_list

# 执行
dic = load_dictionary()
fully_segment("商品和服务",dic)

['商', '商品', '品', '和', '和服', '服', '服务', '务']

In [38]:
# 正向最长匹配
def forward_segment(text,dic):
    word_list = []
    i = 0 
    while i < len(text):
        longest_word = text[i]
        for j in range(i+1,len(text) + 1):
            word = text[i:j]
            if word in dic:
                if len(word) > len(longest_word):
                    longest_word = word
        word_list.append(longest_word)
        i += len(longest_word)
    return word_list

string = "研究生命起源"
forward_segment(string,dic)

['研究生', '命', '起源']

In [41]:
# 反向最长匹配
def backward_segment(text,dic):
    word_list = []
    i = len(text) - 1
    while i >= 0:
        longest_word = text[i]
        for j in range(0,i):
            word = text[j:i + 1]
            if word in dic:
                if len(word) > len(longest_word):
                    longest_word = word
        print(word_list)
        word_list.insert(0,longest_word)
        i -= len(longest_word)
    return word_list

backward_segment(string,dic)

[]
['起源']
['生命', '起源']


['研究', '生命', '起源']

In [46]:
# 双向最长匹配
def count_single_char(word_list:list):
    return sum(1 for word in word_list if len(word) == 1)

def bidirectional_segment(text,dic):
    f = forward_segment(text,dic)
    b = backward_segment(text,dic)
    if len(f) < len(b):
        return f
    elif len(f) > len(b):
        return b
    else:
        if count_single_char(f) < count_single_char(b):
            return f
        else:
            return b
        
bidirectional_segment(string,dic)

[]
['起源']
['生命', '起源']


['研究', '生命', '起源']

In [43]:
def evaluate_speed(segment,text,dic):
    start_time = time.time()
    for i in range(pressure):
        segment(text,dic)

In [65]:
# 想要加速这个词典模型？散列表太慢了，必须用自己构造的字典树。
# 构建一颗字典树：
class Node(object):
    def __init__(self,value) -> None:
        # 二叉树的节点
        self.children = {}
        self._value = value
        
    def _add_child(self,char,value,overwrite = False):
        # 添加节点
        child = self._children.get(char)
        if child is None:
            child = Node(value)
            self._children[char] = child
        elif overwrite:
            child._value = value
        return child

In [67]:
class Trie(Node):
    def __init__(self) -> None:
        super().__init__(None)
        
    def __contains__(self,key):
        return self[key] is not None
    
    def __getitem__(self,key):
        state = self